# Gene set enrichment analysis of rho scores
- tool: blitzgsea, https://github.com/MaayanLab/blitzgsea

In [1]:
import numpy as np 
import pandas as pd
import anndata as ad
import blitzgsea as blitz

In [2]:
import screenpro

In [3]:
from matplotlib import font_manager as fm

In [4]:
from matplotlib import rcParams

In [5]:
font_files = fm.findSystemFonts(fontpaths=None, fontext='ttf')

for font_file in font_files:
    fm.fontManager.addfont(font_file)

In [6]:
# {f.name for f in matplotlib.font_manager.fontManager.ttflist}

In [7]:
rcParams['font.family'] = ['Arial']

In [8]:
from screenpro.load import loadScreenProcessingData, read_adata_pkl

In [9]:
pager_dir = "/data_gilbert/home/aarab/tools/pager/"
pager_annotation_path = '/data_gilbert/home/aarab/tools/pager/annotations/human'

In [10]:
c5_gobp_gmt = blitz.enrichr.read_gmt(
    f'{pager_annotation_path}/msigdb_v7.4_c5.go.bp/c5.go.bp.v7.4.symbols.gmt'
)

In [11]:
def get_sig_gsea_results(result,variable,threshold):
    return result[result[variable] < threshold]

def get_rho_for_gsea(rho):
    
    rho = rho.loc[~rho.label.eq('pseudo'),:]

    rho_score = rho.score.reset_index()
    rho_score = rho_score.dropna()
    
    rho_stat =  1 - rho.pvalue # regardless of direction
    rho_stat = rho_stat.reset_index()
    rho_stat = rho_stat.dropna()
    
    return rho_score, rho_stat

# def run_rho_gsea(rho,gmt,min_size=200):
#     signature = rho.copy()

#     result = blitz.gsea(
#         signature=signature,
#         library=gmt,
#         min_size=min_size,
#         # max_size=250,
#         verbose=True
#     )
    
#     return signature, result

def run_rho_gsea_directional(rho,gmt,min_size=15,max_size=150):
    signature = rho.copy()

    result = blitz.gsea(
        signature=signature,
        library=gmt,
        min_size=min_size,
        max_size=max_size,
        verbose=True
    )
    
    return signature, result

In [12]:
# import sys
# import pandas as pd
# from itertools import chain, product

# sys.path.append("../../")

# from scripts.util import *
# from matplotlib_venn import venn2
# from IPython.display import IFrame

In [13]:
# import gseapy

### Get phenotype scores

In [14]:
Ci_adata = read_adata_pkl('datasets/CRISPRi')
Ca_A549_adata = read_adata_pkl('datasets/CRISPRa_A549')
Ca_k562_adata = read_adata_pkl('datasets/CRISPRa_k562')

In [15]:
def get_annotated_score_df(adata,score,level,threshold):
    if level == 'transcript':
        df = adata.transcript_scores[score]['ave_rep1_rep2'][[
            'average phenotype of strongest 3',
            'Mann-Whitney p-value',
        ]].reset_index('gene').reset_index(drop=True).copy()
    
    elif level == 'gene':
        df = adata.gene_scores[score]['ave_rep1_rep2'][[
            'average phenotype of strongest 3',
            'Mann-Whitney p-value',
        ]].reset_index('gene').copy()
        
    df = screenpro.phenoScore.ann_score_df(df, ctrl_label='pseudo',threshold=threshold)
    
    return df

In [16]:
screenpro.phenoScore.ann_score_df

<function screenpro.phenoScore.ann_score_df(df_in, up_hit='resistance_hit', down_hit='sensitivity_hit', ctrl_label='non-targeting', threshold=10)>

In [17]:
threshold = 2

In [18]:
rho_dict = dict([
    (treat,get_annotated_score_df(Ci_adata,score,'gene',threshold = threshold).set_index('target')) for treat, score in Ci_adata.comparisons.items()
])

In [19]:
rho_df = pd.concat(dict([(treat,rho.score) for treat, rho in rho_dict.items()]),axis=1)
# rho_df = pd.concat(rho_dict,axis=1)

rho_label = pd.concat(dict([(treat,rho.label) for treat, rho in rho_dict.items()]),axis=1)

In [20]:
rho_bin = pd.DataFrame(0,index=rho_label.index,columns=rho_label.columns)

rho_bin = (rho_label.isin(['sensitivity_hit']) * -1) + (rho_label.isin(['resistance_hit']) * 1)

# rho_bin = rho_bin[~rho_bin.eq(0).sum(axis=1).eq(9)]

## Run GSEA

In [21]:
    # gmt = blitz.enrichr.read_gmt(
    #     f'{pager_annotation_path}/msigdb_v7.4_c5.go/c5.go.v7.4.symbols.gmt',
    #     background_genes = rho_score.target.to_list(),
    # )

    # result = blitz.gsea(
    #     signature=signature,
    #     library=gmt,
    #     min_size=10,
    #     max_size=150,
    #     processes = 10,
    #     verbose=True
    # )

In [22]:
results = {}
gmt = c5_gobp_gmt

for treat, rho in rho_dict.items():
    print(treat)
    rho_score, _ = get_rho_for_gsea(rho)

    signature, result = run_rho_gsea_directional(rho_score,c5_gobp_gmt)

    result_top = get_sig_gsea_results(result,'fdr',0.2)
    
    results[treat] = {'signature':signature,'full':result,'top':result_top} #,'gmt':gmt
    
    del signature,result,result_top,rho_score

PARPi


Enrichment : 100%|██████████| 7481/7481 [00:02<00:00, 3340.54it/s]
/data_gilbert/home/aarab/anaconda3/envs/mamba/envs/screenpro2/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))


DNAPKi


Enrichment : 100%|██████████| 7481/7481 [00:02<00:00, 2902.33it/s]
/data_gilbert/home/aarab/anaconda3/envs/mamba/envs/screenpro2/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))


ATMi


...

Enrichment : 100%|██████████| 7481/7481 [00:02<00:00, 3199.75it/s]
/data_gilbert/home/aarab/anaconda3/envs/mamba/envs/screenpro2/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))


PARPi+ATRi


Enrichment : 100%|██████████| 7481/7481 [00:02<00:00, 3175.28it/s]
/data_gilbert/home/aarab/anaconda3/envs/mamba/envs/screenpro2/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))


PARPi+WEE1i


Enrichment : 100%|██████████| 7481/7481 [00:02<00:00, 3247.18it/s]
/data_gilbert/home/aarab/anaconda3/envs/mamba/envs/screenpro2/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))


### save as excel files

In [23]:
writer = pd.ExcelWriter('results/go_gsea.xlsx', engine='openpyxl')

for name,res in results.items():
    res['top'].to_excel(writer, sheet_name=f'CRISPRi {name}')
    
# writer.save()
writer.close()

In [24]:
pd.concat(dict([
    (k, results[k]['top'][results[k]['top'].leading_edge.str.contains('PRDX1')])
    for k in results.keys()
])).to_excel('results/go_with_prdx1.xlsx')

In [25]:
all_terms = [res['full'].index.to_list() for name,res in results.items()]    
ol_terms  = list(set(all_terms[0]).intersection(*all_terms[1:]))

In [26]:
pd.concat(dict([(name,res['full'].loc[ol_terms,:]) for name,res in results.items()]),axis=1).to_excel('results/go_gsea_merged.xlsx')

### GSEA plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def save_image(figs,filename):
    
    # PdfPages is a wrapper around pdf 
    # file so there is no clash and create
    # files with no error.
    p = PdfPages(filename)
      
    # iterating over the numbers in list
    for fig in figs: 
        
        # and saving the files
        fig.savefig(p, format='pdf',bbox_inches='tight') 
      
    # close the object
    p.close()  

#### sensitivity_genesets

In [ ]:
fig_list = []

for treat in results.keys():
    fig = blitz.plot.top_table(
        results[treat]['signature'],
        results[treat]['gmt'],
        results[treat]['top'].query('nes < -2')
    )

    fig.suptitle(treat,fontsize=20)
    
    fig_list.append(fig)

save_image(fig_list,'go_gsea_top_sensitivity_genesets.pdf')

#### resistance_genesets

In [ ]:
fig_list = []

for treat in results.keys():
    fig = blitz.plot.top_table(
        results[treat]['signature'],
        results[treat]['gmt'],
        results[treat]['top'].query('nes > 2')
    )

    fig.suptitle(treat,fontsize=20)
    
    fig_list.append(fig)

save_image(fig_list,'go_gsea_top_resistance_genesets.pdf')

# 

In [ ]:
g = sns.clustermap(
    rho_bin.loc[list(set(c5_go_gmt['GOBP_CELL_REDOX_HOMEOSTASIS']) & set(rho_bin.index)),:],
    cmap='RdBu_r',
    vmin=-2,
    vmax=2,
    figsize=(4,12),
)

g.fig.suptitle('GOBP CELL REDOX HOMEOSTASIS',size=16) 
g.cax.set_visible(False)

plt.tight_layout()
g.savefig('GOBP_CELL_REDOX_HOMEOSTASIS.pdf')
plt.show()

# 

In [26]:
from watermark import watermark
print(
    watermark()
)
print('_'*80)
print(
    watermark(iversions=True, globals_=globals())
)

Last updated: 2023-07-27T15:55:37.892224-07:00

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.14.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 3.10.0-957.27.2.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit

________________________________________________________________________________
matplotlib: 3.7.2
screenpro : 0.2.1
blitzgsea : 1.3.34
pandas    : 2.0.3
anndata   : 0.9.1
numpy     : 1.24.4

